In [0]:
%python
from pyspark.sql.functions import sum

# Load the sessions.csv file with an absolute path
sessions_df = spark.read.csv("/Volumes/internal_1/default/online_gaming/sessions.csv", header=True, inferSchema=True)

# ------------------------------------------------
# 1. Calculate Total In-Game Purchases Per Player
# ------------------------------------------------
print("Calculating Total In-Game Purchases Per Player...")

# Filter for purchase tasks
purchase_df = sessions_df.filter(sessions_df["task"] == "purchase")

# Group by player_id and sum the amount_spent
total_purchases_per_player = purchase_df.groupBy("player_id") \
                                     .agg(sum("amount_spent($)").alias("total_purchases($)"))

# Display the results
print("Total in-game purchases per player:")
display(total_purchases_per_player)

Calculating Total In-Game Purchases Per Player...
Total in-game purchases per player:


player_id,total_purchases($)
P033,169.31
P019,113.35
P030,116.24
P034,117.68
P060,29.83
P076,58.03
P001,41.39
P068,155.63
P040,443.09000000000003
P022,76.12


Databricks visualization. Run in Databricks to view.

question:2

In [0]:
%python
from pyspark.sql.functions import to_timestamp, col, max

# ------------------------------------------------
# 2. Compute Max Session Length Per Player
# ------------------------------------------------
print("\nCalculating Maximum Session Length Per Player...")

# Convert string timestamps to PySpark timestamps
sessions_with_timestamps = sessions_df.withColumn(
    "login_timestamp",
    to_timestamp("login_time", "dd-MM-yyyy HH:mm")
).withColumn(
    "logout_timestamp",
    to_timestamp("logout_time", "dd-MM-yyyy HH:mm")
)

# Calculate session length in minutes
sessions_with_length = sessions_with_timestamps.withColumn(
    "session_length_minutes",
    (col("logout_timestamp").cast("long") - col("login_timestamp").cast("long")) / 60
)

# Group by player_id and find the maximum session length
max_session_length_per_player = sessions_with_length.groupBy("player_id") \
                                                     .agg(max("session_length_minutes").alias("max_session_length_minutes"))

# Display the results
print("Maximum session length per player:")
display(max_session_length_per_player)


Calculating Maximum Session Length Per Player...
Maximum session length per player:


player_id,max_session_length_minutes
P045,30.0
P033,170.0
P063,104.0
P075,14.0
P041,13.0
P019,179.0
P029,32.0
P030,121.0
P034,100.0
P008,20.0


Databricks visualization. Run in Databricks to view.

In [0]:
%python
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, max, col, to_timestamp

# Create a SparkSession
spark = SparkSession.builder.appName("GamingDataAnalysis").getOrCreate()

# Load the sessions.csv file
sessions_df = spark.read.csv("/Volumes/internal_1/default/online_gaming/sessions.csv", header=True, inferSchema=True)

# Calculate total purchases per player
purchase_df = sessions_df.filter(sessions_df.task == "purchase")
total_purchases_per_player = purchase_df.groupBy("player_id").agg(sum("amount_spent($)").alias("total_purchases($)"))
print("Total in-game purchases per player (DataFrame):")
total_purchases_per_player.show()

# Calculate max session length per player
sessions_with_timestamps = sessions_df.withColumn(
    "login_timestamp", to_timestamp("login_time", "dd-MM-yyyy HH:mm")
).withColumn(
    "logout_timestamp", to_timestamp("logout_time", "dd-MM-yyyy HH:mm")
)
sessions_with_length = sessions_with_timestamps.withColumn(
    "session_length_minutes",
    (col("logout_timestamp").cast("long") - col("login_timestamp").cast("long")) / 60
)
max_session_length_per_player = sessions_with_length.groupBy("player_id").agg(max("session_length_minutes").alias("max_session_length_minutes"))
print("Maximum session length per player (DataFrame):")
max_session_length_per_player.show()

Total in-game purchases per player (DataFrame):
+---------+------------------+
|player_id|total_purchases($)|
+---------+------------------+
|     P033|            169.31|
|     P019|            113.35|
|     P030|            116.24|
|     P034|            117.68|
|     P060|             29.83|
|     P076|             58.03|
|     P001|             41.39|
|     P068|            155.63|
|     P040|443.09000000000003|
|     P022|             76.12|
|     P055|            113.91|
|     P026|            135.32|
|     P027|            163.69|
|     P044|             93.85|
|     P004|             15.83|
|     P064|            116.78|
+---------+------------------+

Maximum session length per player (DataFrame):
+---------+--------------------------+
|player_id|max_session_length_minutes|
+---------+--------------------------+
|     P045|                      30.0|
|     P033|                     170.0|
|     P063|                     104.0|
|     P075|                      14.0|
|     P041|

In [0]:
%python
sessions_df.createOrReplaceTempView("sessions")

In [0]:
%python
top_spenders = spark.sql("""
    SELECT
        player_id,
        SUM(`amount_spent($)`) AS total_spend
    FROM
        sessions
    WHERE
        task = 'purchase'
    GROUP BY
        player_id
    ORDER BY
        total_spend DESC
    LIMIT 5
""")
display(top_spenders)

player_id,total_spend
P040,443.09000000000003
P033,169.31
P027,163.69
P068,155.63
P026,135.32


Databricks visualization. Run in Databricks to view.

In [0]:
%python
churn_rate = spark.sql("""
    WITH LastMonthPlayers AS (
        SELECT
            DISTINCT player_id
        FROM
            sessions
        WHERE
            task = 'login' AND
            MONTH(to_timestamp(login_time, 'dd-MM-yyyy HH:mm')) = MONTH(CURRENT_DATE()) - 1 AND
            YEAR(to_timestamp(login_time, 'dd-MM-yyyy HH:mm')) = YEAR(CURRENT_DATE())
    ),
    ThisMonthPlayers AS (
        SELECT
            DISTINCT player_id
        FROM
            sessions
        WHERE
            task = 'login' AND
            MONTH(to_timestamp(login_time, 'dd-MM-yyyy HH:mm')) = MONTH(CURRENT_DATE()) AND
            YEAR(to_timestamp(login_time, 'dd-MM-yyyy HH:mm')) = YEAR(CURRENT_DATE())
    ),
    ChurnedPlayers AS (
        SELECT
            l.player_id
        FROM
            LastMonthPlayers l
        LEFT ANTI JOIN
            ThisMonthPlayers t ON l.player_id = t.player_id
    )
    SELECT
        COUNT(player_id) AS churned_players_count,
        (COUNT(player_id) * 100.0) / (SELECT COUNT(*) FROM LastMonthPlayers) AS churn_rate_percentage
    FROM
        ChurnedPlayers
""")
display(churn_rate)

churned_players_count,churn_rate_percentage
24,100.00000000000000
